# Load Data

Load sales data from S3 / HDFS. We use the built-in "csv" method, which can use the first line has column names and which also supports infering the schema automatically. We use both and save some code for specifying the schema explictly.

We also peek inside the data by retrieving the first five records.

In [21]:
from pyspark.sql.functions import *

raw_data = spark.read\
    .option("header","true")\
    .option("inferSchema","true")\
    .csv("s3://dimajix-training/data/kc-house-data")

raw_data.limit(5).toPandas()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## Inspect Schema

Now that we have loaded the data and that the schema was inferred automatically, let's inspect it.

In [22]:
raw_data.printSchema()

root
 |-- id: long (nullable = true)
 |-- date: string (nullable = true)
 |-- price: decimal(7,0) (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: double (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- condition: integer (nullable = true)
 |-- grade: integer (nullable = true)
 |-- sqft_above: integer (nullable = true)
 |-- sqft_basement: integer (nullable = true)
 |-- yr_built: integer (nullable = true)
 |-- yr_renovated: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- sqft_living15: integer (nullable = true)
 |-- sqft_lot15: integer (nullable = true)



## Split training / validation set

In [23]:
# Split the data - 80% for training, 20% for validation
(training_data, validation_data) = raw_data.randomSplit([0.8,0.2])

print("training_data = " + str(training_data.count()))
print("validation_data = " + str(validation_data.count()))

training_data = 17291
validation_data = 4322


## Adding more Features

The RMSE tells us that on average our prediction actually performs pretty bad. How can we improve that? Obviously we used only the size of the house for the price prediction so far, but we have a whole lot of additional information. So let's make use of that. The mathematical idea is that we create a more complex (but still linear) model that also includes other features.

Let's recall that a linear  model looks as follows:

    y = SUM(coeff[i]*x[i]) + intercept
    
This means that we are not limited to single feature `x`, but we can use many features `x[0]...x[n]`. Let's do that with the house data!

### Inspect data

Since we don't have any additional information, we model some of the features differently. So far we used all features as direct linear predictors, which implies that a grade of 4 is twice as good as 2. Maybe that is not the case and not all predictors have a linear influence. Specifically nominal and ordinal features should be modeled differntly as categories. More an that later.

First let's have a look at the data agin using Spark `describe`

In [24]:
raw_data.describe().toPandas()

,summary,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,count,21613,21613,21613,21613,21613,21613,21613,21613,21613,...,21613,21613,21613,21613,21613,21613,21613,21613,21613,21613
1,mean,4.580301520864988E9,None,540088.1418,3.37084162309721,2.1147573219821405,2079.8997362698374,15106.967565816869,1.4943089807060566,0.007541757275713691,...,7.656873178179799,1788.3906907879516,291.5090454818859,1971.0051357978994,84.40225790033776,98077.93980474715,47.56005251931704,-122.21389640494083,1986.552491556008,12768.455651691113
2,stddev,2.8765655713120522E9,None,367127.19648270035,0.930061831147451,0.770163157217741,918.4408970468096,41420.51151513551,0.5399888951423489,0.08651719772788748,...,1.1754587569743344,828.0909776519175,442.57504267746685,29.373410802386243,401.67924001917504,53.505026257472466,0.13856371024192368,0.14082834238139288,685.3913042527788,27304.179631338524
3,min,1000102,20140502T000000,75000,0,0.0,290,520,1.0,0,...,1,290,0,1900,0,98001,47.1559,-122.519,399,651
4,max,9900000190,20150527T000000,7700000,33,8.0,13540,1651359,3.5,1,...,13,9410,4820,2015,2015,98199,47.7776,-121.315,6210,871200


In [25]:
raw_data.select(countDistinct(col("zipcode"))).toPandas()

,count(DISTINCT zipcode)
0,70


## New Features using One-Hot Encoding

A simple but powerful method for creating new features from categories (i.e. nominal and ordinal features) is to use One-Hot-Encoding. For each nominal feature, the set of all possible values is indexed from 0 to some n. But since it cannot be assumed that larger values for n have a larger impact, a different approach is chosen. Instead each possible values is encoded by a 0/1 vector with only a single entry being one.

Lets try that with the tools Spark provides to us.

### Indexing Nominal Data
First we need to index the data. Since Spark cannot know, which or how many distinct values are present in a specific column, the `StringIndexer` works like a ML algorithm: First it needs to be fit to the data, thereby returning an `StringIndexerModel` which then can be used for transforming data.

Let's perform both steps and let us look at the result

In [26]:
from pyspark.ml.feature import *

indexer = StringIndexer() \
    .setInputCol("zipcode") \
    .setOutputCol("zipcode_idx") \
    .setHandleInvalid("keep")
    
index_model = indexer.fit(training_data)    
indexed_zip_data = index_model.transform(training_data)

indexed_zip_data.limit(10).toPandas()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,zipcode_idx
0,1000102,20140916T000000,280000,6,3.00,2400,9373,2.0,0,0,...,2400,0,1991,0,98002,47.3262,-122.214,2060,7316,52.0
1,1000102,20150422T000000,300000,6,3.00,2400,9373,2.0,0,0,...,2400,0,1991,0,98002,47.3262,-122.214,2060,7316,52.0
2,1200019,20140508T000000,647500,4,1.75,2060,26036,1.0,0,0,...,1160,900,1947,0,98166,47.4444,-122.351,2590,21891,47.0
3,1200021,20140811T000000,400000,3,1.00,1460,43000,1.0,0,0,...,1460,0,1952,0,98166,47.4434,-122.347,2250,20023,47.0
4,3600057,20150319T000000,402500,4,2.00,1650,3504,1.0,0,0,...,760,890,1951,2013,98144,47.5803,-122.294,1480,3504,22.0
5,3600072,20150330T000000,680000,4,2.75,2220,5310,1.0,0,0,...,1170,1050,1951,0,98144,47.5801,-122.294,1540,4200,22.0
6,3800008,20150224T000000,178000,5,1.50,1990,18200,1.0,0,0,...,1990,0,1960,0,98178,47.4938,-122.262,1860,8658,45.0
7,5200087,20140709T000000,487000,4,2.50,2540,5001,2.0,0,0,...,2540,0,2005,0,98108,47.5423,-122.302,2360,6834,55.0
8,6200017,20141112T000000,281000,3,1.00,1340,21336,1.5,0,0,...,1340,0,1945,0,98032,47.4023,-122.273,1340,37703,63.0
9,7200179,20141016T000000,150000,2,1.00,840,12750,1.0,0,0,...,840,0,1925,0,98055,47.4840,-122.211,1480,6969,43.0


### One-Hot-Encoder

Now we have a single number (the index of the value) in a new column `zipcode_idx`. But in order to use the information in a linear model, we need to create sparse vectors from this index with only exactly one `1`. This can be done with the `OneHotEncoder` transformer. This time no fitting is required, the class can be used directly with its `transform` method.

In [27]:
encoder = OneHotEncoder() \
    .setInputCol("zipcode_idx") \
    .setOutputCol("zipcode_onehot")

encoded_zip_data = encoder.transform(indexed_zip_data)
encoded_zip_data.limit(10).toPandas()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,zipcode_idx,zipcode_onehot
0,1000102,20140916T000000,280000,6,3.00,2400,9373,2.0,0,0,...,0,1991,0,98002,47.3262,-122.214,2060,7316,52.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1000102,20150422T000000,300000,6,3.00,2400,9373,2.0,0,0,...,0,1991,0,98002,47.3262,-122.214,2060,7316,52.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1200019,20140508T000000,647500,4,1.75,2060,26036,1.0,0,0,...,900,1947,0,98166,47.4444,-122.351,2590,21891,47.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1200021,20140811T000000,400000,3,1.00,1460,43000,1.0,0,0,...,0,1952,0,98166,47.4434,-122.347,2250,20023,47.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,3600057,20150319T000000,402500,4,2.00,1650,3504,1.0,0,0,...,890,1951,2013,98144,47.5803,-122.294,1480,3504,22.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,3600072,20150330T000000,680000,4,2.75,2220,5310,1.0,0,0,...,1050,1951,0,98144,47.5801,-122.294,1540,4200,22.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,3800008,20150224T000000,178000,5,1.50,1990,18200,1.0,0,0,...,0,1960,0,98178,47.4938,-122.262,1860,8658,45.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,5200087,20140709T000000,487000,4,2.50,2540,5001,2.0,0,0,...,0,2005,0,98108,47.5423,-122.302,2360,6834,55.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,6200017,20141112T000000,281000,3,1.00,1340,21336,1.5,0,0,...,0,1945,0,98032,47.4023,-122.273,1340,37703,63.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,7200179,20141016T000000,150000,2,1.00,840,12750,1.0,0,0,...,0,1925,0,98055,47.4840,-122.211,1480,6969,43.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# Creating Pipelines

Since it would be tedious to add all features one after another and apply a full chain of transformations to the training set, the validation set and eventually to new data, Spark provides a `Pipeline` abstraction. A Pipeline simply contains a sequence of Transformations and (possibly multiple) machine learning algorithms. The whole pipeline then can be trained using the `fit` method which will return a `PipelineModel` instance. This instance contains all transformers and trained models and then can be used directly for prediction.

In [28]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import *
from pyspark.ml.regression import *

pipeline = Pipeline(stages = [
    # For every nominal feature, you have to create a pair of StringIndexer and OneHotEncoder. 
    # The StringIndexer should store its index result in some new column, which then is used 
    # by the OneHotEncoder to create a one-hot vector.
    StringIndexer(
        inputCol = "bathrooms",
        outputCol = "bathrooms_idx",
        handleInvalid = "keep"),
    OneHotEncoder(
        inputCol = "bathrooms_idx",
        outputCol = "bathrooms_onehot"),
    StringIndexer(
        inputCol = "bedrooms",
        outputCol = "bedrooms_idx",
        handleInvalid = "keep"),
    OneHotEncoder(
        inputCol = "bedrooms_idx",
        outputCol = "bedrooms_onehot"),
    StringIndexer(
        inputCol = "floors",
        outputCol = "floors_idx",
        handleInvalid = "keep"),
    OneHotEncoder(
        inputCol = "floors_idx",
        outputCol = "floors_onehot"),
    OneHotEncoder(
        inputCol = "view",
        outputCol = "view_onehot"),
    OneHotEncoder(
        inputCol = "condition",
        outputCol = "condition_onehot"),
    StringIndexer(
        inputCol = "grade",
        outputCol = "grade_idx",
        handleInvalid = "keep"),
    OneHotEncoder(
        inputCol = "grade_idx",
        outputCol = "grade_onehot"),
    StringIndexer(
        inputCol = "zipcode",
        outputCol = "zipcode_idx",
        handleInvalid = "keep"),
    OneHotEncoder(
        inputCol = "zipcode_idx",
        outputCol = "zipcode_onehot"),
    VectorAssembler(
        inputCols = ["bedrooms_onehot", "bathrooms_onehot", "sqft_living", "sqft_lot", "floors_onehot", "waterfront", "view_onehot", "condition_onehot", "grade_onehot", "sqft_above", "sqft_basement", "yr_built", "yr_renovated", "zipcode_onehot", "sqft_living15", "sqft_lot15"],
        outputCol = "features"),
    LinearRegression(
        featuresCol = "features",
        labelCol = "price")
    ]
)


### Train model with training data

Once you created the `Pipeline`, you can fit it in a single step using the `fit` method. This will return an instance of the class `PipelineModel`. Assign this model instace to a value called `model`.

And remember: Use the training data for fitting!

In [29]:
model = pipeline.fit(training_data)

## Evaluate model using validation data

Now that we have a model, we need to measure its performance. This requires that predictions are created by applying the model to the validation data by using the `transform` method of the moodel. The quality metric of the prediction is implemented in the `RegressionEvaluator` class from the Spark ML evaluation package. Create an instance of the evaluator and configure it appropriately to use the column `price` as the target (label) variable and the column `prediction` (which has been created by the pipeline model) as the prediction column. Also remember to set the metric name to `rmse`. Finally feed in the predicted data into the evaluator, which in turn will calculate the desired quality metric (RMSE in our case).

In [31]:
from pyspark.ml.evaluation import *

# Create and configure a RegressionEvaluator
evaluator = RegressionEvaluator(
    labelCol = "price",
    predictionCol = "prediction",
    metricName = "rmse")
    
# Create predictions of the validationData by using the "transform" method of the model
pred = model.transform(validation_data)

# Now measure the quality of the prediction by using the "evaluate" method of the evaluator
rmse = evaluator.evaluate(pred)

print("RMSE = " + str(rmse))

RMSE = 145536.42702738565


# Adding more models

Another way of improving the overall prediction is to add multiple models to a single Pipeline. Each downstream ML algorithm has access to the prediction of the previous stages. This way we can create two independant models and eventually fit a mixed model as the last step. In this example we want to use a simple linear model created by a `LinearRegression` and combine that model with a Poisson model created by a `GeneralizedLinearRegression`. The results of both models eventually are combined using a final `LinearRegression` model.

In [33]:
pipeline = Pipeline(stages = [
    StringIndexer(
        inputCol = "bathrooms",
        outputCol = "bathrooms_idx",
        handleInvalid = "keep"),
    OneHotEncoder(
        inputCol = "bathrooms_idx",
        outputCol = "bathrooms_onehot"),
    StringIndexer(
        inputCol = "bedrooms",
        outputCol = "bedrooms_idx",
        handleInvalid = "keep"),
    OneHotEncoder(
        inputCol = "bedrooms_idx",
        outputCol = "bedrooms_onehot"),
    StringIndexer(
        inputCol = "floors",
        outputCol = "floors_idx",
        handleInvalid = "keep"),
    OneHotEncoder(
        inputCol = "floors_idx",
        outputCol = "floors_onehot"),
    OneHotEncoder(
        inputCol = "view",
        outputCol = "view_onehot"),
    OneHotEncoder(
        inputCol = "condition",
        outputCol = "condition_onehot"),
    StringIndexer(
        inputCol = "grade",
        outputCol = "grade_idx",
        handleInvalid = "keep"),
    OneHotEncoder(
        inputCol = "grade_idx",
        outputCol = "grade_onehot"),
    StringIndexer(
        inputCol = "zipcode",
        outputCol = "zipcode_idx",
        handleInvalid = "keep"),
    OneHotEncoder(
        inputCol = "zipcode_idx",
        outputCol = "zipcode_onehot"),
    VectorAssembler(
        inputCols = ["bedrooms_onehot", "bathrooms_onehot", "sqft_living", "sqft_lot", "floors_onehot", "waterfront", "view_onehot", "condition_onehot", "grade_onehot", "sqft_above", "sqft_basement", "yr_built", "yr_renovated", "zipcode_onehot", "sqft_living15", "sqft_lot15"],
        outputCol = "features"),
    LinearRegression(
        featuresCol = "features",
        labelCol = "price",
        predictionCol = "linear_prediction"),
    GeneralizedLinearRegression(
        featuresCol = "features",
        labelCol = "price",
        family = "poisson",
        link = "log",
        predictionCol = "poisson_prediction"),
    VectorAssembler(
        inputCols = ["linear_prediction","poisson_prediction"],
        outputCol = "pred_features"),
    LinearRegression(
        featuresCol = "pred_features",
        labelCol = "price",
        predictionCol = "prediction")
    ]
)

### Train model with training data

Again as usual we train a model using the `fit` method of the pipeline.

In [35]:
model = pipeline.fit(training_data)

### Evaluate model using validation data

And eventually we measure the performance of the combined model by using the evaluator created some steps above.

In [36]:
# First create predictions by applying the learnt pipeline model to the validation data
pred = model.transform(validation_data)

# And now calculate the performance metric by using the evaluator on the predictions
rmse = evaluator.evaluate(pred)

print("RMSE = " + str(rmse))

RMSE = 125240.39350934756


### Inspect Model

Let us inspect the coefficients of the last step, which tells us which of both models (linear or poisson) has more weight.

In [39]:
model.stages[len(model.stages)-1].coefficients

DenseVector([0.1726, 0.8274])